In [ ]:
from nnsight import LanguageModel
import torch

from dictionary_learning import ActivationBuffer
from dictionary_learning.training import trainSAE
from circuits.nanogpt_to_hf_transformers import NanogptTokenizer, convert_nanogpt_model
from dictionary_learning.utils import hf_dataset_to_generator
from dictionary_learning.trainers.standard import StandardTrainer

In [ ]:
DEVICE = torch.device("cuda")

tokenizer = NanogptTokenizer()
model = convert_nanogpt_model("lichess_8layers_ckpt_no_optimizer.pt", torch.device(DEVICE))
model = LanguageModel(model, device_map=DEVICE, tokenizer=tokenizer)

submodule = model.transformer.h[5].mlp  # layer 1 MLP
activation_dim = 512  # output dimension of the MLP
dictionary_size = 8 * activation_dim

batch_size = 8

data = hf_dataset_to_generator("adamkarvonen/chess_sae_test", streaming=False)
buffer = ActivationBuffer(
    data,
    model,
    submodule,
    n_ctxs=512,
    ctx_len=256,
    refresh_batch_size=4,
    io="out",
    d_submodule=512,
    device=DEVICE,
    out_batch_size=batch_size,
)

In [ ]:
from dictionary_learning import AutoEncoder

ae = AutoEncoder.from_pretrained("t1_ae.pt", device=DEVICE)

In [ ]:
from dictionary_learning.evaluation import evaluate

eval_results = evaluate(ae, buffer, device=DEVICE, batch_size=64)

In [ ]:
for k, v in eval_results.items():
    print(k,v)